In [1]:
import os
import json
import time
import torch
import argparse
import numpy as np
from multiprocessing import cpu_count
from torch.utils.data import DataLoader
from collections import OrderedDict, defaultdict
import torch.nn as nn
from torch.autograd import Variable

from ptb import PTB
from utils import to_var, idx2word, expierment_name
from model import SentenceVAE

In [2]:
ts = time.strftime('%Y-%b-%d-%H-%M-%S', time.gmtime())

In [3]:
save_model_path = os.path.join('won', ts)
os.makedirs('.\\'+save_model_path)

In [4]:
datasets = OrderedDict()

In [5]:
splits = ['train', 'valid'] + (['test'] if False else [])

In [6]:
splits

['train', 'valid']

In [7]:
for split in splits:
    datasets[split] = PTB(
        data_dir='data',
        split=split,
        create_data=False,
        max_sequence_length=60,
        min_occ=1
    )

In [8]:
vocab_size = datasets['train'].vocab_size

In [9]:
sos_idx=datasets['train'].sos_idx

In [10]:
eos_idx=datasets['train'].eos_idx

In [11]:
pad_idx=datasets['train'].pad_idx

In [12]:
max_sequence_length = 60
embedding_size = 300
hidden_size = 256
word_dropout = 0.5
latent_size = 16
num_layers = 1
bidirectional = True
batch_size_fit = 32
rnn_type = 'rnn'
learning_rate = 0.001
k = 0.0025
x0 = 2500

In [13]:
class RVAE(nn.Module):
    def __init__(self,vocab_size, embedding_size, max_sequence_length, hidden_size, word_dropout, latent_size,
                sos_idx, eos_idx, pad_idx, rnn_type='rnn' , num_layers=1, bidirectional=True):
        
        super().__init__()
        
        self.max_sequence_length = max_sequence_length
        self.sos_idx = sos_idx
        self.eos_idx = eos_idx
        self.pad_idx = pad_idx
        
        self.latent_size = latent_size
        self.rnn_type = rnn_type
        self.bidirectional = bidirectional
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        
        #self.outputs2vocab = nn.Linear(hidden_size * (2 if bidirectional else 1), vocab_size)
        self.encoder = Encoder(vocab_size = vocab_size,embedding_size = embedding_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional = bidirectional,latent_size = latent_size,rnn_type = rnn_type).cuda()
        self.decoder = Decoder(vocab_size = vocab_size,embedding_size = embedding_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional = bidirectional,latent_size = latent_size,rnn_type = rnn_type).cuda()
    
    def forward(self,x,length):
        
        mu,logvar,reparam = self.encoder(Variable(x),length)
        logp  = self.decoder(Variable(x),reparam)
        
        
        return logp, mu, logvar, reparam

In [14]:
class Encoder(nn.Module):
    def __init__(self,vocab_size,embedding_size, hidden_size, latent_size, bidirectional=True, num_layers = 1,rnn_type='rnn'):
        super(Encoder,self).__init__()
        
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.latent_size = latent_size
        self.rnn_type = rnn_type
        self.bidirectional = bidirectional
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        if self.rnn_type == 'rnn':
            rnn = nn.RNN
        elif self.rnn_type == 'gru':
            rnn = nn.GRU
        elif self.rnn_type =='lstm':
            rnn = nn.LSTM
        else:
            raise ValueError()
            
        self.embedding = nn.Embedding(vocab_size,embedding_size)
        self.encoder = rnn(self.embedding_size, self.hidden_size, num_layers = self.num_layers, bidirectional = self.bidirectional, batch_first = True)
        
        self.hidden_factor = (2 if self.bidirectional else 1) * self.num_layers
        
        self.hidden2mean = nn.Linear(self.hidden_size* self.hidden_factor, self.latent_size)
        self.hidden2logv = nn.Linear(self.hidden_size* self.hidden_factor, self.latent_size)
                        
    
    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        
        eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps).cuda()
        
        return eps.mul(std).add_(mu)
    
    def forward(self,x,length):
        batch_size = x.size(0)
        sorted_lengths, sorted_idx = torch.sort(length, descending=True)
        input_sequence = x[sorted_idx.cuda()]
        #print(input_sequence)
        _,hidden = self.encoder(self.embedding(input_sequence))
        if self.bidirectional or self.num_layers > 1:
            # flatten hidden state
            hidden = hidden.view(batch_size, self.hidden_size*self.hidden_factor)
        else:
            hidden = hidden.squeeze()

        mu = self.hidden2mean(hidden)
        logvar = self.hidden2logv(hidden)

        reparam = self.reparametrize(mu,logvar)
        
        return mu,logvar,reparam
        


In [15]:
class Decoder(nn.Module):
    def __init__(self,vocab_size,embedding_size, hidden_size, latent_size, bidirectional=True, num_layers = 1,rnn_type='rnn',word_dropout = 0.5):
        super(Decoder,self).__init__()
        
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.latent_size = latent_size
        self.rnn_type = rnn_type
        self.bidirectional = bidirectional
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        if self.rnn_type == 'rnn':
            rnn = nn.RNN
        elif self.rnn_type == 'gru':
            rnn = nn.GRU
        elif self.rnn_type =='lstm':
            rnn = nn.LSTM
        else:
            raise ValueError()
            
        self.hidden_factor = (2 if self.bidirectional else 1) * self.num_layers            
        self.latent2hidden = nn.Linear(latent_size, hidden_size * self.hidden_factor)
        
        self.embedding = nn.Embedding(vocab_size,embedding_size)
        self.word_dropout = nn.Dropout(p=word_dropout)
        
        self.decoder = rnn(embedding_size, hidden_size, num_layers=num_layers, bidirectional=self.bidirectional, batch_first=True)
        self.outputs2vocab = nn.Linear(hidden_size * (2 if bidirectional else 1), vocab_size)
        
    def forward(self,x,z):
        
        batch_size = x.size(0)
        sorted_lengths, sorted_idx = torch.sort(length, descending=True)
        input_sequence = x[sorted_idx.cuda()]
        
        hidden = self.latent2hidden(z)
        if self.bidirectional or self.num_layers > 1:
            # unflatten hidden state
            hidden = hidden.view(self.hidden_factor, batch_size, self.hidden_size)
        else:
            hidden = hidden.unsqueeze(0)
            
        outputs,_ = self.decoder(self.embedding(input_sequence),hidden)
        
        logp =nn.functional.log_softmax(self.outputs2vocab(outputs))
        
        return logp
        


In [24]:
rvae=RVAE(vocab_size, embedding_size, max_sequence_length, hidden_size, word_dropout, latent_size,sos_idx, eos_idx, pad_idx)

In [25]:
def kl_anneal_function(anneal_function, step, k, x0):
    if anneal_function == 'logistic':
        return float(1/(1+np.exp(-k*(step-x0))))
    elif anneal_function == 'linear':
        return min(1, step/x0)

NLL = torch.nn.NLLLoss(size_average=False, ignore_index=datasets['train'].pad_idx)

In [26]:
def loss_fn(logp, target, length, mean, logv, anneal_function, step, k, x0):

    # cut-off unnecessary padding from target, and flatten
    target = target[:, :60].contiguous().view(-1)
    logp = logp.view(-1, logp.size(2))

    # Negative Log Likelihood
    NLL_loss = NLL(logp, target)

    # KL Divergence
    KL_loss = -0.5 * torch.sum(1 + logv - mean.pow(2) - logv.exp())
    KL_weight = kl_anneal_function(anneal_function, step, k, x0)

    return NLL_loss, KL_loss, KL_weight

In [27]:
optimizer = torch.optim.Adam(rvae.parameters(), lr=learning_rate)

In [28]:
print(rvae)

RVAE(
  (outputs2vocab): Linear(in_features=512, out_features=9877, bias=True)
  (encoder): Encoder(
    (embedding): Embedding(9877, 300)
    (encoder): RNN(300, 256, batch_first=True, bidirectional=True)
    (hidden2mean): Linear(in_features=512, out_features=16, bias=True)
    (hidden2logv): Linear(in_features=512, out_features=16, bias=True)
  )
  (decoder): Decoder(
    (latent2hidden): Linear(in_features=16, out_features=512, bias=True)
    (embedding): Embedding(9877, 300)
    (word_dropout): Dropout(p=0.5)
    (decoder): RNN(300, 256, batch_first=True, bidirectional=True)
    (outputs2vocab): Linear(in_features=512, out_features=9877, bias=True)
  )
)


In [29]:
epochs = 10

In [31]:
step = 0
avg_losses = []
losses = []
NLL_losses = []
KL_losses = []
KL_weights = []

for epoch in range(epochs):
    for split in splits:
        data_loader = DataLoader(
                    dataset=datasets[split],
                    batch_size=batch_size_fit,
                    shuffle=split=='train',
                    num_workers=cpu_count(),
                    pin_memory=torch.cuda.is_available()
                )

        if split == 'train':
            rvae.train()
        else:
            rvae.eval()
            
        for iteration, batch in enumerate(data_loader):
            batch_size = batch['input'].size(0)
            
            optimizer.zero_grad()
            
            x = batch['input'].type(torch.cuda.LongTensor)
            length = batch['length']
            
            logp, mean, logv, z=rvae(x,length)
            
            NLL_loss, KL_loss, KL_weight = loss_fn(logp, Variable(batch['target']).type(torch.cuda.LongTensor),batch['length'], mean, logv, 'logistic', step, k, x0)
            
            loss = (NLL_loss + KL_weight * KL_loss)/batch_size
            
            losses.append(float(loss.cpu().data))
            NLL_losses.append(NLL_loss.data[0]/batch_size)
            KL_losses.append(KL_loss.data[0]/batch_size)
            KL_weights.append(KL_weight)
            

            
            if split == 'train':
                loss.backward()
                optimizer.step()
                step += 1

            if iteration % 100 == 0 or iteration+1 == len(data_loader):
                print("%s Batch %04d/%i, Loss %9.4f, NLL-Loss %9.4f, KL-Loss %9.4f, KL-Weight %6.3f"
            %(split.upper(), iteration, len(data_loader)-1, loss.data[0], NLL_loss.data[0]/batch_size, KL_loss.data[0]/batch_size, KL_weight))
                np.savez(L=losses,file='loss.npz')
                np.savez(L=NLL_losses,file='NLL_losses.npz')
                np.savez(L=KL_losses,file='KL_losses.npz') 
                np.savez(L=KL_weights,file='KL_weights.npz')
                
         # save checkpoint
        if split == 'train':
            checkpoint_path = os.path.join(save_model_path, "E%i.pytorch"%(epoch))
            torch.save(rvae.state_dict(), checkpoint_path)
            print("Model saved at %s"%checkpoint_path)
        print("%s Epoch %02d/%i, Mean ELBO %9.4f"%(split.upper(), epoch, epochs, np.mean(np.array(losses))))
        avg_losses.append(np.mean(np.array(losses)))
        np.savez(L=avg_losses,file='avg_losses.npz')

C:\Users\asd13\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:46: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


TRAIN Batch 0000/1314, Loss   87.2974, NLL-Loss   87.2956, KL-Loss    0.9128, KL-Weight  0.002
TRAIN Batch 0100/1314, Loss   73.6464, NLL-Loss   73.6410, KL-Loss    2.1666, KL-Weight  0.002
TRAIN Batch 0200/1314, Loss   85.0424, NLL-Loss   85.0384, KL-Loss    1.2654, KL-Weight  0.003
TRAIN Batch 0300/1314, Loss   86.8569, NLL-Loss   86.8531, KL-Loss    0.9415, KL-Weight  0.004
TRAIN Batch 0400/1314, Loss   81.6852, NLL-Loss   81.6788, KL-Loss    1.2377, KL-Weight  0.005
TRAIN Batch 0500/1314, Loss   73.4471, NLL-Loss   73.4420, KL-Loss    0.7675, KL-Weight  0.007
TRAIN Batch 0600/1314, Loss   81.3431, NLL-Loss   81.3404, KL-Loss    0.3150, KL-Weight  0.009
TRAIN Batch 0700/1314, Loss   75.1611, NLL-Loss   75.1556, KL-Loss    0.4995, KL-Weight  0.011
TRAIN Batch 0800/1314, Loss   75.6880, NLL-Loss   75.6834, KL-Loss    0.3261, KL-Weight  0.014
TRAIN Batch 0900/1314, Loss   94.4389, NLL-Loss   94.4341, KL-Loss    0.2635, KL-Weight  0.018
TRAIN Batch 1000/1314, Loss   79.2838, NLL-Loss   

TRAIN Batch 0900/1314, Loss   75.1706, NLL-Loss   75.1696, KL-Loss    0.0010, KL-Weight  1.000
TRAIN Batch 1000/1314, Loss   79.1760, NLL-Loss   79.1729, KL-Loss    0.0031, KL-Weight  1.000
TRAIN Batch 1100/1314, Loss   84.6929, NLL-Loss   84.6924, KL-Loss    0.0005, KL-Weight  1.000
TRAIN Batch 1200/1314, Loss   82.2200, NLL-Loss   82.2193, KL-Loss    0.0007, KL-Weight  1.000
TRAIN Batch 1300/1314, Loss   82.1853, NLL-Loss   82.1842, KL-Loss    0.0012, KL-Weight  1.000
TRAIN Batch 1314/1314, Loss   65.6020, NLL-Loss   65.6011, KL-Loss    0.0009, KL-Weight  1.000
Model saved at won\2018-May-10-08-38-57\E4.pytorch
TRAIN Epoch 04/10, Mean ELBO   79.0946
VALID Batch 0000/105, Loss   81.0051, NLL-Loss   81.0043, KL-Loss    0.0008, KL-Weight  1.000
VALID Batch 0100/105, Loss   61.6299, NLL-Loss   61.6291, KL-Loss    0.0008, KL-Weight  1.000
VALID Batch 0105/105, Loss   52.2723, NLL-Loss   52.2716, KL-Loss    0.0008, KL-Weight  1.000
VALID Epoch 04/10, Mean ELBO   79.0803
TRAIN Batch 0000/13

TRAIN Batch 0000/1314, Loss   74.5312, NLL-Loss   74.5195, KL-Loss    0.0117, KL-Weight  1.000
TRAIN Batch 0100/1314, Loss   75.5702, NLL-Loss   75.5700, KL-Loss    0.0002, KL-Weight  1.000
TRAIN Batch 0200/1314, Loss   80.4788, NLL-Loss   80.4704, KL-Loss    0.0083, KL-Weight  1.000
TRAIN Batch 0300/1314, Loss   78.6548, NLL-Loss   78.6541, KL-Loss    0.0007, KL-Weight  1.000
TRAIN Batch 0400/1314, Loss   77.2497, NLL-Loss   77.2496, KL-Loss    0.0001, KL-Weight  1.000
TRAIN Batch 0500/1314, Loss   74.8573, NLL-Loss   74.8557, KL-Loss    0.0015, KL-Weight  1.000
TRAIN Batch 0600/1314, Loss   79.1657, NLL-Loss   79.1654, KL-Loss    0.0002, KL-Weight  1.000
TRAIN Batch 0700/1314, Loss   89.2990, NLL-Loss   89.2989, KL-Loss    0.0001, KL-Weight  1.000
TRAIN Batch 0800/1314, Loss   77.2643, NLL-Loss   77.2638, KL-Loss    0.0005, KL-Weight  1.000
TRAIN Batch 0900/1314, Loss   74.4014, NLL-Loss   74.3337, KL-Loss    0.0676, KL-Weight  1.000
TRAIN Batch 1000/1314, Loss   83.2737, NLL-Loss   

In [32]:
from utils import to_var, idx2word, interpolate

with open('data'+'/ptb.vocab.json', 'r') as file:
    vocab = json.load(file)

w2i, i2w = vocab['w2i'], vocab['i2w']

samples = torch.topk(nn.functional.softmax(rvae.outputs2vocab(outputs)),1,dim=-1)[1].squeeze()

samples = samples.cpu().data.numpy()

sent_str = [str()]*len(samples)

for i, sent in enumerate(samples):
    for word_id in sent:
        if word_id == w2i['<pad>']: 
            break
        sent_str[i] += i2w[str(word_id)] + " "
    sent_str[i] = sent_str[i].strip()

samples

NameError: name 'outputs' is not defined